In [1]:
import sys
import json
sys.path.append(".")
from my_constant import *

In [2]:
# path
common_data_path = '../egogoal/v2/annotations/'
ego4d_json_path = "../ego4d_data/ego4d.json"
goalstep_train_path = common_data_path + "goalstep_train.json"
goalstep_test_path = common_data_path + "goalstep_test_unannotated.json"
goalstep_val_path = common_data_path + "goalstep_val.json"

# read files
with open(ego4d_json_path, "r") as file:
    ego4d_json = json.load(file)
with open(goalstep_train_path, "r") as file:
    goalstep_train_json = json.load(file)
with open(goalstep_val_path, "r") as file:
    goalstep_val_json = json.load(file)
with open(goalstep_test_path, "r") as file:
    goalstep_test_json = json.load(file)

In [3]:
# scenario_type = []
# for video in video_list:
#     scenario_type.extend(video["scenarios"])
# frequency = Counter(scenario_type)
# scenario_type = [item for item, count in frequency.most_common()]
# print(scenario_type)

scenario_type = ['Cooking', 'Baker', 'Cleaning / laundry', 'Talking with family members', 'Eating', 'Talking with friends/housemates', 'Watching tv', "BBQ'ing/picnics", 'On a screen (phone/laptop)', 'Preparing hopot', 'Playing with pets', 'Listening to music', 'Farmer', 'Sleeping', 'Hosting a party', 'Playing games / video games', 'Household management - caring for kids', 'Making a salad/sandwich', 'Indoor Navigation (walking)', 'Play with cellphone', 'Household cleaners', 'Reading books', 'Daily hygiene', 'Working at desk', 'Doing yardwork / shoveling snow', 'Assembling a puzzle', 'Crafting/knitting/sewing/drawing/painting', 'Dancing', 'Street art', 'Bus', 'Talking to colleagues', 'Working in outdoor store', 'Camp setup/pack-up/chores', 'Car mechanic', 'Skateboard/scooter', 'Swimming in a pool/ocean', 'Grocery shopping indoors']

print(len(scenario_type))

37


In [4]:
def find_video_by_uid(uid, video_list):
    return next((item for item in video_list[Component.videos] if item[Component.video_uid] == uid), None)

def fill_json_using_split_data(video_json, split_data):
    video_json[Component.goal_category] = split_data[Component.goal_category]
    video_json[Component.goal_description] = split_data[Component.goal_description]
    for segment_lev2 in split_data[Component.segments]:
        new_segment_lev2 = copy.deepcopy(segment_format_lev2)
        for segment_lev3 in segment_lev2[Component.segments]:
            new_segment_lev3 = copy.deepcopy(segment_format_lev3)
            new_segment_lev2[Component.segments].append(new_segment_lev3)
        video_json[Component.segments].append(new_segment_lev2)

def replace_slash_with_dash(input_string):
    if "/" in input_string:
        return input_string.replace("/", "-")
    else:
        return input_string

In [ ]:
#input filter values
min_sec = input("minimum duration in secs(default: 0): ")
if min_sec == "":
    min_sec = 0
else:
    min_sec = int(min_sec)
max_sec = input("maximum duration in secs(default: 100000000): ")
if max_sec == "":
    max_sec = 100000000
else:
    max_sec = int(max_sec)

print(f"min sec = {min_sec}")
print(f"max sec = {max_sec}")

In [11]:
start = 20
end = start+5
min_sec = 0
max_sec = 100000000

#input filter values
scenario_num = input("write scenario counts(max:37)\nCooking:1\nBaker:2\nCleaning/laundry:3\nTalking with family members:4\nEating:5")
if scenario_num == "":
    scenario_num = 1
else:
    scenario_num = int(scenario_num)

scenario = scenario_type[scenario_num -1]

#apply filters
#check goalsteps first
video_list = [] # 851 for goalstep benchmarks
for video in ego4d_json[Component.videos]:
    if video["split_goalstep"] is not None:
        video_list.append(video)

scenario_list = []
for video in video_list:
    if scenario in video[Component.scenarios]:
        scenario_list.append(video)

duration_list = []
for video in scenario_list:
    if min_sec <= video[Component.duration_sec] < max_sec:
        duration_list.append(video)

uid_list = []
for video in duration_list:
    uid_list.append(video[Component.video_uid])
    
print(f"we find {len(uid_list)} uids")

#save duration_list and uid list
#can also save by splits(train and only val useful)
new_annotations = copy.deepcopy(annotation_format)
for uid in uid_list:
    video_info = find_video_by_uid(uid, ego4d_json)
    
    # json for save
    video_json = copy.deepcopy(video_format)
    video_json[Component.video_uid] = uid
    video_json[Component.duration] = video_info[Component.duration_sec]
    video_json[Component.scenarios] = video_info[Component.scenarios]
    
    additional_video_info = find_video_by_uid(uid, goalstep_train_json)
    if additional_video_info != None:
        video_json[Component.split] = "train"
        fill_json_using_split_data(video_json, additional_video_info)
        new_annotations[Component.videos].append(video_json)
    else:
        additional_video_info = find_video_by_uid(uid, goalstep_val_json)
        if additional_video_info != None:
            video_json[Component.split] = "val"
            fill_json_using_split_data(video_json, additional_video_info)
            new_annotations[Component.videos].append(video_json)
        # if you want to contain test data into annotations, uncomment the line below
        # else:
        #     additional_video_info = find_video_data(uid, goalstep_test_json)
        #     if additional_video_info != None:
        #         video_json[Component.split] = "test"
        #         # No Additional Pre-Info from test data
        #     else:
        #         # Strange Situation
        #         print(f"{uid} is nowhere in tran/val/test");
sorted_data = sorted(new_annotations[Component.videos], key=lambda x: x[Component.duration])

data_number = 0
for data in sorted_data:
    data[Component.number] = data_number
    segment_number_lev2 = 0
    for segment_lev2 in data[Component.segments]:
        segment_lev2[Component.number] = segment_number_lev2
        segment_number_lev3 = 0
        for segment_lev3 in segment_lev2[Component.segments]:
            segment_lev3[Component.number] = segment_number_lev3
            segment_number_lev3 += 1
        segment_number_lev2 += 1
    data_number += 1
    
# new_file_path = f"{annotation_data_path}{scenario}/{replace_slash_with_dash(scenario)}.json"
# with open(new_file_path, "w") as json_file:
#     json.dump({Component.videos: sorted_data}, json_file, indent=4)
#new_file_top5_path = f"{annotation_data_path}{replace_slash_with_dash(scenario)}/{replace_slash_with_dash(scenario)}_top{start+1}-{end}.json"
new_file_top5_path = f"{replace_slash_with_dash(scenario)}_top{start+1}-{end}.json"

with open(new_file_top5_path, "w") as json_file:
    json.dump({Component.videos: sorted_data[start:end]}, json_file, indent=4)

we find 85 uids


In [ ]:
# Data 형식 수정할 때 필요한 코드 - 20241230 SDH
# level 다음에 각 segment ID를 써주는 형식으로 수정하는 코드이다.
import re

def process_file(input_file, output_file):
    # 파일 읽기
    with open(input_file, "r") as file:
        lines = file.readlines()

    # 데이터 처리
    processed_lines = []
    for line in lines:
        # "number": "0-7" 패턴 찾기
        match = re.search(r'"number":\s*"(.*?)"', line)
        if match:
            # 문자열을 숫자 리스트로 분리
            numbers = [int(x) for x in match.group(1).split('-')]
            # 가장 큰 숫자로 변경
            last_number = numbers[-1]
            # 라인 수정
            line = re.sub(r'"number":\s*"(.*?)"', f'"number": {last_number}', line)
        processed_lines.append(line)

    # 결과 저장
    with open(output_file, "w") as file:
        file.writelines(processed_lines)

# 사용 예시
filenames = ['Baker_top5.json', 'Baker.json', 'Cleaning - laundry_top5.json', 'Cleaning - laundry.json', 'Cooking_top5.json', 'Cooking.json', 'Eating_top5.json', 'Eating.json', 'Talking with family members_top5.json', 'Talking with family members.json']
for filename in filenames:
    process_file(f"annotations/{filename}", f"fixed/{filename}")